### Итоговое задание Александра Соколова
### по Проекту 7. Ford vs Ferrari: определяем модель авто по фото
####   Юнит 8. Нейронные сети (отредактирован 13.01.2021)
---

# 1. Импорты библиотек, инициализация глобальных констант, контроль инициализации пакетов и оборудования
## 1.1. Импорты библиотек

In [ ]:
import os
from datetime import datetime as dt
import numpy as np
import pandas as pd
import math
import sys
import random as rn

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import Sequential as S
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3
from keras import backend as K

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [ ]:
import utils_module09122020 as utils

## 1.2. Глобальные константы

In [ ]:
# CURRENT_DIR = './'  # имя текущей директории для локальной машины 
CURRENT_DIR = '../'  # имя текущей директории для каггл

# проверка пути в папке input для корректного назначения глобальной константы PATH_TO_FILE
print(os.listdir(CURRENT_DIR+'input/'))

In [ ]:
PATH_to_FILE = CURRENT_DIR+'input/sf-dl-car-classification/'  # имя директории с исходными файлами
PATH_to_WORKDIR = CURRENT_DIR+'working/'  # имя рабочей директории
PATH_to_FILE_RESULT = '../input/last-result/last_result.csv'
PATH_to_FILE_descr = '../input/last-result/descr_result.pkl'

RANDOM_SEED = 42  # фиксируем состояние генератора псевдо-случайных чисел для воспроизводимости результата
np.random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = '0'
rn.seed(RANDOM_SEED)

!pip3 freeze > requirements.txt  # фиксируем версии всех пакетов для воспроизводимости результата

CURRENT_DATE = dt.now().strftime('[%d.%m.%Y]')  # фиксируем текущую дату для контроля версий экспериментов

## 1.3. Проверка подключения и параметров GPU

In [ ]:
# утилита проверяет подключение GPU
utils.check_GPU_ON()

# 2. Импорт и предобработка данных
## 2.1. Импорт данных

In [ ]:
df_train = pd.read_csv(PATH_to_FILE + 'train.csv')
df_submit = pd.read_csv(PATH_to_FILE + 'sample-submission.csv')

## 2.2. Предобработка данных

In [ ]:
# создание в рабочей папке отдельной папки для разъархивированных картинок авто
PATH_to_IMAGES = utils.mkdir(PATH_to_WORKDIR, 'images_of_car')

In [ ]:
utils.unzip(PATH_to_FILE, 
            PATH_to_IMAGES, 
            ['train.zip', 'test.zip'], 
            'с картинками авто')

In [ ]:
# создаем дополнительные константы для путей к тренировочным и тестовым фото
PATH_to_IMAGES_TRAIN = PATH_to_IMAGES + 'train/'
PATH_to_IMAGES_TEST = PATH_to_IMAGES + 'test_upload/'

# 3. Разведовательный анализ данных (EDA)
## 3.1. Статистический анализ

In [ ]:
df_train.head()

In [ ]:
print('Краткая информация о тренировочном датасете')
utils.describe_without_plots_all_collumns(df_train, 
                                          short=True)

In [ ]:
print('Посмотрим какие категории представлены:')
df_train.Category.unique()

In [ ]:
utils.simple_plot_barv('Распределение кол-ва картинок (с % откл. от среднего)', 
                       'Category', 
                       df_train, 
                       1.3, 
                       'Категория', 
                       'Кол-во картинок в категории авто')

In [ ]:
df_submit.head()

In [ ]:
print('Краткая информация о датасете с шаблоном сабмишена')
utils.describe_without_plots_all_collumns(df_submit, 
                                          short=True)

#### 3.2. Визуальный анализ картинок

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категориям (случайная выборка по кат.)', 
                                    'Категория:',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    range(10), 
                                    'Id', 
                                    RANDOM_SEED, 
                                    True)

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 0 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.zeros(10, dtype=int), 
                                    'Id', 
                                    RANDOM_SEED+2, 
                                    False)

0 категория = Lada Priora

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 1 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int), 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

1 категория = Ford Focus

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 2 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*2, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

2 категория = Lada Samara 2114

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 3 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*3, 
                                    'Id', 
                                    RANDOM_SEED+1, 
                                    False)

3 категория = Lada 110

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 4 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*4, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

4 категория = Lada 2107

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 5 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*5, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

5 категория - Lada Niva

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 6 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*6, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

6 категория - Lada Kalina

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 7 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*7, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

7 категория - Lada Samara 2109

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 8 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*8, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

8 категория - Volkswagen Passat

In [ ]:
utils.images_from_dataset_with_path('Пример картинок по категории = 9 (случайная выборка по кат.)', 
                                    '',
                                    PATH_to_IMAGES_TRAIN, 
                                    df_train, 
                                    'Category', 
                                    np.ones(10, dtype=int)*9, 
                                    'Id', 
                                    RANDOM_SEED, 
                                    False)

9 категория - Lada Samara 21099

Резюме: 
- Для классификации представлены фотографии 10 категорий авто Lada
- Категории достаточно сбалансированны по кол-ву фото
- Размеры фотографий различаются, но в основном 640 на 480. Цветные.
- Всего 22236 фото в том числе:15561 фото в трейне и 6675 в тесте
- Кол-во фото в трейне не достаточно для хорошего обучения модели нейронной сети, поэтому будем применять различные виды аугментации данных

# 4. Инициализация датасета для хранения ВСЕХ! гиперпараметров и результатов экспериментов
Создание и обучение модели включает в себя несколько этапов:
- аугментацию данных (AUG)
- загрузку основы для модели (M)
- создание архитектуры головы (H)
- компиляция модели (C)
- предварительное обучение модели - это этап многочисленных небольших по продолжительности экспериментов для определения предварительных оптимальных параметров для обучения модели
- дообучение модели Fine-tuning  

На каждом этапе нам потребуются какие-то гиперпараметры. К сожалению сохранение модели в формате h5(hdf5) сохраняет только частично информацию с этапов (M) и (H) с обученными весами модели. Мне оказалось этого не достаточно для анализа оптимальных гиперпараметров поэтому я решил создать датасет с результатами (R) экспериментов.

In [ ]:
results_of_exp = pd.DataFrame()  # датафрейм для хранения гиперпараметров и результатов экспериментов
descr_hyperp_of_exp = {}  # словарь для хранения описания гиперпараметров

# загружаем результаты предыдущих экспериментов в датасет, 
# в словарь загружаем описание параметров экспериметов
# инициализируем новый эксперимент
NUM_EXP, \
results_of_exp, \
descr_hyperp_of_exp = utils.load_result_last_cnn_fit(PATH_to_FILE_RESULT, 
                                                     PATH_to_FILE_descr)

# Строка ниже закомментирована, так как используется только в первый запуск ноутбука, 
# когда результатов экспериментов нет
# NUM_EXP = utils.hyperp('NUM_EXP',1,'Номер эксперимента',results_of_exp,descr_hyperp_of_exp)  # инициализация всех гиперпараметров происходит через специальный метод

# 5. Предварительная обработка фотографий для обучения модели (аугментация)

In [ ]:
print('Инициализация гиперпараметров для Аугментации фото:')
AUG_ROTATION_RANGE = utils.hyperp('AUG_ROTATION_RANGE', 
                                  5, 
                                  'Аугментация. Диапазон для случайных поворотов в градусах',
                                  results_of_exp,
                                  descr_hyperp_of_exp)
AUG_BRIGHTNES_RANGE = utils.hyperp('AUG_BRIGHTNES_RANGE', 
                                   [0.5, 1.5], 
                                   'Аугментация. Диапазон выбора значения сдвига яркости',
                                   results_of_exp,
                                   descr_hyperp_of_exp)
AUG_WIDTH_SHIFT_RANGE = utils.hyperp('AUG_WIDTH_SHIFT_RANGE', 
                                     0.1, 
                                     'Аугментация. Диапазон выбора значения сдвига ширины',
                                     results_of_exp,
                                     descr_hyperp_of_exp)
AUG_HEIGHT_SHIFT_RANGE = utils.hyperp('AUG_HEIGHT_SHIFT_RANGE', 
                                      0.1, 
                                      'Аугментация. Диапазон выбора значения сдвига высоты',
                                      results_of_exp,
                                      descr_hyperp_of_exp)
AUG_HORIZONTAL_FLIP = utils.hyperp('AUG_HORIZONTAL_FLIP', 
                                   True, 
                                   'Аугментация. Логическое значение для произвольного  переворачивания по горизонтали',
                                   results_of_exp,
                                   descr_hyperp_of_exp)
AUG_RESCALE = utils.hyperp('AUG_RESCALE', 
                           1. / 255, 
                           'Аугментация. Коэффициент масштабирования. Мы умножаем данные на это значение (после применения всех других преобразований)',
                           results_of_exp,
                           descr_hyperp_of_exp)
AUG_VAL_SPLIT = utils.hyperp('AUG_VAL_SPLIT', 
                             0.15, 
                             'Аугментация. Доля выборки оставляемой для валидации вне тренировочной выборки',
                             results_of_exp,
                             descr_hyperp_of_exp)
AUG_IMG_SIZE = utils.hyperp('AUG_IMG_SIZE', 
                            224, 'Аугментация. Размер картинки, которую будем подавать в нейросеть',
                            results_of_exp,
                            descr_hyperp_of_exp)
AUG_BATCH_SIZE = utils.hyperp('AUG_BATCH_SIZE', 
                              64, 
                              'Аугментация. Размер Batch',
                              results_of_exp,
                              descr_hyperp_of_exp)

In [ ]:
print('Инициализация константы типа Аугментации')
AUG_TYPE = utils.hyperp('AUG_TYPE', 
                        'ImageDataGenerator', 
                        'Аугментация. Тип',
                        results_of_exp,
                        descr_hyperp_of_exp)

In [ ]:
# Генерация аугментированных данных
train_generator, \
valid_generator, \
test_generator = utils.train_valid_test_generators(
                    AUG_ROTATION_RANGE,           
                    AUG_BRIGHTNES_RANGE,
                    AUG_WIDTH_SHIFT_RANGE, 
                    AUG_HEIGHT_SHIFT_RANGE,
                    AUG_HORIZONTAL_FLIP,
                    AUG_RESCALE,
                    AUG_VAL_SPLIT,
                    PATH_to_IMAGES_TRAIN,
                    PATH_to_IMAGES_TEST,
                    df_submit,
                    AUG_IMG_SIZE,
                    AUG_BATCH_SIZE,
                    RANDOM_SEED)

# 6. Создание и предобучение модели нейросети
## 6.1 Создание модели

In [ ]:
print('Инициализация константы типа базовой модели')
M_BASE_TYPE = utils.hyperp('M_BASE_TYPE', 
                           'Xception', 
                           'Модель. Тип базовой модели',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# используем перенос обучения (Transfer learning)
# поэтому сначала загружаем предобученную базовую модель Xception без головы так как будем ставить свою
base_model = Xception(
                    weights='imagenet', 
                    include_top=False, 
                    input_shape = (AUG_IMG_SIZE, AUG_IMG_SIZE, 3))

In [ ]:
print('Инициализация констант архитектуры головы и компиляции модели') 
H_DROPOUT_RATE =  utils.hyperp('H_DROPOUT_RATE', 
                               0.25, 
                               'Голова модели. Вероятность отключения нейронов в слое Dropout',
                               results_of_exp,
                               descr_hyperp_of_exp)
H_USE_BIAS =  utils.hyperp('H_USE_BIAS', 
                           'Default', 
                           'Голова модели. Использование коэф Байес в полносвязном слое',
                           results_of_exp,
                           descr_hyperp_of_exp)
H_KERNEL_REG = utils.hyperp('H_KERNEL_REG', 
                            'Default', 
                            'Голова модели. Использование специальной регуляризации в полносвязном слое',
                            results_of_exp,
                            descr_hyperp_of_exp)
H_BATCHNORM =  utils.hyperp('H_BATCHNORM', 
                            False, 
                            'Голова модели. Использование BatchNormalization' ,
                            results_of_exp,
                            descr_hyperp_of_exp)

C_LR = utils.hyperp('C_LR', 
                    1e-4, 
                    'Компиляция модели. Шаг обучения модели',
                    results_of_exp,
                    descr_hyperp_of_exp)
С_OPTIMIZER_TYPE = utils.hyperp('С_OPTIMIZER_TYPE', 
                                'Adam', 
                                'Компиляция модели. Тип оптимайзера',
                                results_of_exp,
                                descr_hyperp_of_exp)
C_LOSS_TYPE = utils.hyperp('C_LOSS_TYPE', 
                           'categorical_crossentropy', 
                           'Компиляция модели. Тип функции потерь (loss-функции)',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# архитектура головы (классификационные слои поверх базовой модели)
head = S([GlobalAveragePooling2D(), 
          Dense(256,activation='relu'),
          Dropout(H_DROPOUT_RATE),
          Dense(10, activation='softmax')])
# обратите внимание, что тут повторяется архитектура baseline 
# только без функционального программирования


# Собираем модель используя подход перенос обучения (Transfer learning)
model = utils.model_assembler(base_model, head)

# Компилируем модель
model.compile(loss=C_LOSS_TYPE, 
              optimizer=optimizers.Adam(lr=C_LR), 
              metrics=['accuracy'])

In [ ]:
# выводим краткую информацию о моделе
# стандартное саммари очень громоздкая портянка вниз и смотреть ее утомительно
# я нашел несколько визуализаторов нейросетей с достаточно полной информацией о слоях и функциях
# но они не корректно отображаются в каггл
# если вы знаете удобные способы визуализации моеделей, 
# тогда напишите пожалуйста мне или в комментариях к ноутбуку
utils.model_summary_short(model, 
                          base_model,
                          ' (из baseline)')

## 6.2 Обучение модели

In [ ]:
print('Инициализация констант обучения модели (из baseline)') 
M_EPOCHS = utils.hyperp('M_EPOCHS', 
                        5, 
                        'Модель. Кол-во эпох обучения модели',
                        results_of_exp,
                        descr_hyperp_of_exp)

M_CALLBACKS_TYPE = utils.hyperp('M_CALLBACKS_TYPE', 
                                'MC_T', 
                                'Модель. Тип callback: MC - ModelCheckpoint, ES - EarlyStopping, LRS - LearningRateScheduler, T - Time Callbacks',
                                results_of_exp,
                                descr_hyperp_of_exp)
M_EPOCHS_DROP = utils.hyperp('M_EPOCHS_DROP', 
                             2, 
                             'Модель. Кол-во эпох через которое LR изменяется внутри функции sheduler',
                             results_of_exp,
                             descr_hyperp_of_exp)
M_LR_UPDATE = utils.hyperp('M_LR_UPDATE', 
                           math.exp(-0.1), 
                           'Модель. Коэфф изменения LR функции sheduler',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# Инициализация Callbacks 
time_cb = utils.TimingCallback()
callbacks_list = utils.callbacks_assembler(M_CALLBACKS_TYPE, 
                                           C_LR, 
                                           M_LR_UPDATE, 
                                           M_EPOCHS_DROP, 
                                           time_cb)

In [ ]:
# код обучения закомментирован для удобства воспроизведения ноутбука
# ниже идет полная загрузка ранее сохраненной модели этого обучения
# и выводится картинка с прошлым процессом обучения для наглядности
# если вы захотите выполнить обучение модели, тогда закомментируйте код 
# загрузки ранее сохраненной лучшей из обученных моделей (строчкой ниже)

# # обучение модели (из baseline)
# right_steps_per_epoch = len(train_generator)

# history = model.fit_generator(train_generator,
#                               steps_per_epoch = right_steps_per_epoch,
#                               validation_data = valid_generator, 
#                               validation_steps = len(valid_generator),
#                               epochs = M_EPOCHS,
#                               callbacks = callbacks_list)

In [ ]:
# загрузка ранее сохраненной модели из baseline
print('Загружена ранее сохраненная лучшая из обученных моделей (из baseline)')
model.load_weights('../input/last-result/best_model_25_20201220__07_47.hdf5')
utils.show_image('../input/last-result/pic_procces_model_fit_baseline_25.png',
                'Скриншот обучения сохраненной модели из base_line', 14, 8, 0.95)
history, time_cb = [], []
model.save('best_model.hdf5')

In [ ]:
# загружаем лучшую модель и получаем значения метрик на валидационной выборке для результатов
print('Загружаем лучшую модель после обучения и проверяем метрики на валидационной выборке:')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
print('Сохраняем метрики на лучшей моделе')
R_EVA_VAL_ACC = utils.hyperp('R_EVA_VAL_ACC', 
                             temp[1], 
                             'Результат обучения. Оценка метрики точности на лучшей моделе',
                             results_of_exp,
                             descr_hyperp_of_exp)
R_EVA_VAL_LOSS = utils.hyperp('R_EVA_VAL_LOSS', 
                             temp[0], 
                             'Результат обучения. Значение функции потерь на лучшей моделе',
                             results_of_exp,
                             descr_hyperp_of_exp)

## 6.3 Сохранение результатов обучения модели

In [ ]:
PATH_to_LAST_results_data = utils.save_model(PATH_to_WORKDIR,
                                             PATH_to_WORKDIR,
                                             model,
                                             history, 
                                             time_cb, 
                                             results_of_exp,
                                             descr_hyperp_of_exp,
                                             True)
# метод записывает результаты обучения модели в датасет с результатами
# также сохраняет дату и время и эксперимента
# загружает в модель лучшую, сохраняет ее с именем номера эксперимента, даты и времени
# после успешного сохранения результатов, формирует два zip-файла и выводит 
# удобные для скачивания ссылки, вы можете сохранить их себе на локальную машину
# не забудьте сделать Refresh рабочей папки 
# или вы можете сохранить два файла с результатами и описаниями гиперпараметров в датасет
# и загружать их при следующем запуске

## 6.4 Результаты обучения модели

In [ ]:
# рисуем графики точности и функции потерь по эпохам
# для удобства на графике номер экперимента, дата, время обучения и оценочная точность по вал выборке
utils.plot_acc_loss_fit_model_in_one (25, 
                                      results_of_exp,
                                      ' (из baseline)')

In [ ]:
# выводим сводную информацию по гиперпараметрам и результатам 
utils.show_result_exp(25,
                      results_of_exp,
                      descr_hyperp_of_exp,
                      ' (baseline)')

***Резюме:***  
Это пример на базе baseline (Transfer learning - Xception) с которого начались многочисленные эксперименты по выбору базовой модели и оптимальных предварительных параметров для того чтобы перейти к тонкой настройке (Fine_Tuning).  

Результатом экспериментов стали предварительные параметры, которые использовались далее в обучении с переносом (Transfer learning). Можно выделить главные моменты по результатам предварительных экпериментов: 
- была выбрана базовая модель EfficientNetB3 
- BatchNormalization в архитектуре головы
- встроенная регуляризация *l2* паралельно со слоем DROPOUT (несмотря на то, что существует много мнений о том, что паралельно эти два момента это оверкил) по факту на этом примере вместе сработали хорошо. Без регуляризации и на региляризации *l1* скор хуже.
- полезный хак - удаление нейронов смещения из слоя (use_bias=False) перед BatchNormalization - позволяет экономить расчетные ресурсы и соответсвенно увеличивать скорость обучения

Эксперименты проводились в отдельном [ноутбуке](https://www.kaggle.com/sokolovaleks/sf-dst-10-car-classification-sokolov-3?scriptVersionId=50603588) для удобства. Кроме этого вы можете посмотреть результаты экспериментов в [датасете на github](https://github.com/alex-sokolov2011/skillfactory_rds/tree/master/module_7) или в [датасете на kaggle](https://www.kaggle.com/sokolovaleks/last-result).


# 7. Transfer learning с Fine-tuning
## 7.1 Этап 1. Замораживаем базовую часть модели полностью

In [ ]:
# инициализируем новый эксперимент копированием гиперпараметров из предыдущего
# так как сесия не останавливалась
print('Инициализируем новый эксперимент Fine-Tuning (Этап 1):')
NUM_EXP = utils.new_exp_without_stop_session(results_of_exp)

In [ ]:
# назначаем новые гиперпараметры для аугментации исходя из оптимальных
# при повторном изменении константы необязательно указывать описание гиперпараметра 
# оно уже сохранено в словаре при инициализации
print('Назначаем новые гиперпараметры для аугментации исходя из оптимальных для Fine-Tuning (Этап 1)')
AUG_ROTATION_RANGE = utils.hyperp('AUG_ROTATION_RANGE', 
                                  10, 
                                  '',
                                  results_of_exp,
                                  descr_hyperp_of_exp)
AUG_IMG_SIZE = utils.hyperp('AUG_IMG_SIZE', 
                            320, 
                            '',
                            results_of_exp,
                            descr_hyperp_of_exp)
AUG_BATCH_SIZE = utils.hyperp('AUG_BATCH_SIZE', 
                              32, 
                              '',
                              results_of_exp,
                              descr_hyperp_of_exp)
AUG_VAL_SPLIT = utils.hyperp('AUG_VAL_SPLIT', 
                             0.05, 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
AUG_RESCALE = utils.hyperp('AUG_RESCALE', 
                           1, 
                           '',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# генерируем новые аугментированные данные
train_generator, \
valid_generator, \
test_generator = utils.train_valid_test_generators(
                    AUG_ROTATION_RANGE,           
                    AUG_BRIGHTNES_RANGE,
                    AUG_WIDTH_SHIFT_RANGE, 
                    AUG_HEIGHT_SHIFT_RANGE,
                    AUG_HORIZONTAL_FLIP,
                    AUG_RESCALE,
                    AUG_VAL_SPLIT,
                    PATH_to_IMAGES_TRAIN,
                    PATH_to_IMAGES_TEST,
                    df_submit,
                    AUG_IMG_SIZE,
                    AUG_BATCH_SIZE,
                    RANDOM_SEED)

In [ ]:
print('Инициализируем новый тип базовой модели для Fine-Tuning (Этап 1)')
M_BASE_TYPE = utils.hyperp('M_BASE_TYPE', 
                           'EfficientNetB3', 
                           '',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# загружаем базовую модель
base_model = EfficientNetB3(
                    weights='imagenet', 
                    include_top=False, 
                    input_shape = (AUG_IMG_SIZE, 
                                   AUG_IMG_SIZE, 
                                   3))

# отключаем обучаемость базовой модели
base_model.trainable = False

In [ ]:
print('Назначаем новые гиперпараметры для головы и компиляции модели для Fine-Tuning (Этап 1)')
H_DROPOUT_RATE =  utils.hyperp('H_DROPOUT_RATE', 
                               0.25, 
                               '',
                               results_of_exp,
                               descr_hyperp_of_exp)
H_USE_BIAS =  utils.hyperp('H_USE_BIAS', 
                           False, 
                           '',
                           results_of_exp,
                           descr_hyperp_of_exp)
H_KERNEL_REG = utils.hyperp('H_KERNEL_REG', 
                            'l2', 
                            '',
                            results_of_exp,
                            descr_hyperp_of_exp)
H_BATCHNORM =  utils.hyperp('H_BATCHNORM', 
                            True, 
                            '',
                            results_of_exp,
                            descr_hyperp_of_exp)
C_LR = utils.hyperp('C_LR', 
                    1e-3, 
                    '',
                    results_of_exp,
                    descr_hyperp_of_exp)
С_OPTIMIZER_TYPE = utils.hyperp('С_OPTIMIZER_TYPE', 
                                'Adam_amsgrad', 
                                '',
                                results_of_exp,
                                descr_hyperp_of_exp)
C_LOSS_TYPE = utils.hyperp('C_LOSS_TYPE', 
                           'categorical_crossentropy', 
                           'Компиляция модели. Тип функции потерь (loss-функции)',
                           results_of_exp,descr_hyperp_of_exp)

In [ ]:
# архитектура головы (классификационные слои поверх базовой модели) 
# для Fine-Tuning (Этап 1)
head = S([GlobalAveragePooling2D(), 
          Dense(128, use_bias=H_USE_BIAS, kernel_regularizer=H_KERNEL_REG),
          BatchNormalization(axis=1), 
          Activation('relu'),
          Dropout(H_DROPOUT_RATE),
          Dense(10, activation='softmax')])


# Собираем модель используя подход перенос обучения (Transfer learning)
model = utils.model_assembler(base_model, 
                              head)

# Компилируем модель
model.compile(loss=C_LOSS_TYPE, 
              optimizer=optimizers.Adam(lr=C_LR, 
                                        amsgrad=True), 
              metrics=['accuracy'])

In [ ]:
# выводим краткое описание модели по параметрам и слоям
utils.model_summary_short(model, 
                          base_model,
                          ' Fine-Tuning (Этап 1)')

In [ ]:
print('Назначаем новые гиперпараметры для модели и обучения для Fine-Tuning (Этап 1)')
M_EPOCHS = utils.hyperp('M_EPOCHS', 
                        15, 
                        '',
                        results_of_exp,
                        descr_hyperp_of_exp)

M_CALLBACKS_TYPE = utils.hyperp('M_CALLBACKS_TYPE', 
                                'MC_ES_LRS_T', 
                                '',
                                results_of_exp,
                                descr_hyperp_of_exp)
M_EPOCHS_DROP = utils.hyperp('M_EPOCHS_DROP', 
                             1, 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
M_LR_UPDATE = utils.hyperp('M_LR_UPDATE', 
                           math.exp(-0.1), 
                           '',
                           results_of_exp,
                           descr_hyperp_of_exp)

In [ ]:
# инициализируем cb переменную и собираем списки cb 
time_cb = utils.TimingCallback()
callbacks_list = utils.callbacks_assembler(M_CALLBACKS_TYPE, 
                                           C_LR, 
                                           M_LR_UPDATE, 
                                           M_EPOCHS_DROP, 
                                           time_cb)

In [ ]:
# код обучения закомментирован для удобства воспроизведения ноутбука
# ниже идет полная загрузка ранее сохраненной модели этого обучения
# и выводится картинка с прошлым процессом обучения для наглядности
# если вы захотите выполнить обучение модели, тогда закомментируйте код 
# загрузки ранее сохраненной лучшей из обученных моделей (строчкой ниже)

# # обучение модели для Fine-Tuning (Этап 1)
# right_steps_per_epoch = len(train_generator)

# history = model.fit_generator(train_generator,
#                               steps_per_epoch = right_steps_per_epoch,
#                               validation_data = valid_generator, 
#                               validation_steps = len(valid_generator),
#                               epochs = M_EPOCHS,
#                               callbacks = callbacks_list)

In [ ]:
# # загрузка ранее выполненной и сохраненной модели Этапа 1 Fine-Tuning
model.load_weights('../input/last-result/best_model_36_20201224__12_36.hdf5')
print('Загружена ранее сохраненная лучшая из обученных моделей Fine-Tuning (Этап 1)')
utils.show_image('../input/last-result/pic_procces_model_fit_ft1_1.png',
                'Скриншоты обучения сохраненной модели Fine-Tuning (Этап 1)', 
                 14, 10,0.92)
utils.show_image('../input/last-result/pic_procces_model_fit_ft1_2.png',
                '', 14, 10, 0.9)
utils.show_image('../input/last-result/pic_procces_model_fit_ft1_3.png',
                '', 14, 10, 0.9)
history, time_cb = [], []
model.save('best_model.hdf5')

In [ ]:
# загружаем лучшую модель и получаем значения метрик на валидационной выборке для результатов
print('Загружаем лучшую модель после обучения и проверяем метрики на валидационной выборке:')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
print('Сохраняем оценку метрики точности и значение функции потерь на лучшей моделе')
R_EVA_VAL_ACC = utils.hyperp('R_EVA_VAL_ACC', 
                             temp[1], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
R_EVA_VAL_LOSS = utils.hyperp('R_EVA_VAL_LOSS', 
                             temp[0], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)

In [ ]:
# сохраняем результаты обучения модели и всю лучшую модель
print('Cохраняем лучшую модель, ее параметры и результаты обучения:')
PATH_to_LAST_results_data = utils.save_model(PATH_to_WORKDIR,
                                             PATH_to_WORKDIR,
                                             model,
                                             history, 
                                             time_cb, 
                                             results_of_exp,
                                             descr_hyperp_of_exp,
                                             True)

In [ ]:
# рисуем графики точности и фнкции потерь по эпохам
# для удобства на графике номер экперимента, дата, время обучения и оценочная точность по вал выборке
utils.plot_acc_loss_fit_model_in_one (36, 
                                      results_of_exp,
                                      ' F-Tuning (Этап 1)')

In [ ]:
# выводим сводную информацию по гиперпараметрам и результатам 
utils.show_result_exp(36,
                      results_of_exp,
                      descr_hyperp_of_exp,
                      ' F-T (Этап 1)')

***Резюме по Fine-Tuning Этап 1:***
- По графику точности валидационной выборки (val_acc сверху над acc) можно сделать вывод, что кол-во эпох подобрано не оптимально и у модели есть большой запас в обучении при увеличении эпох. Но в рамках выполнения первого проекта по глубокому обучению я решил ограничится 15 эпохами, чтобы лучше разобраться во всех инструментах и попытаться выжать максимум на 15 эпохах (и по объему не выше EfficientNetB3 = 48 MB).
- Забегая вперед среднее время обучения одной эпохи на 1 Этапе в среднем (480 сек) против (1650 сек) на дообучении на больших размерах фото, что наводит на мысль что на этом этапе сосредоточен самый важный момент успеха на лб. Рискну предположить, что лучшие результаты сингл моделей на лб были получены на большем кол-ве эпох и более сложных моделях выше EfficientNetB3
- на мой взгляд шаг подобран оптимально, необходимость в плато фиксированного LR я не увидел, так как графики имеют достаточно хороший наклон на данном кол-ве эпох. Замедление тенденции движения линии функции потерь заметно начиная с 14 эпохи, поэтому тем кто решит использовать большее кол-во эпох рекомендую зашить в sheduler плато фиксорованного шага на 2-3 эпохи
- валидация точности методом evalute и внутри метода fit_generator отличается. Но чтобы свести их к единообразию необходимо отказаться от мультисессионности (use_multiprocessing=False), но это увеличивает время обучения примерно на 20%. На этапе тонкой настроки не критично, но забегая вперед на дообучении где общее время обучения 19840 сек. это стало бы проблемой, а мне хотелось сохранить эдентичность гиперпараметров на обучении и на дообучении модели и менять только аугментацию и размер изображения.

## 7.2 Этап 2. Разморозка половины слоев базовой модели

In [ ]:
# инициализируем новый эксперимент копированием гаперпараметров из предыдущего
# так как сесия не останавливалась
print('Инициализируем новый эксперимент Fine-Tuning (Этап 2):')
NUM_EXP = utils.new_exp_without_stop_session(results_of_exp)

In [ ]:
# размораживаем половину базовой модели начиная с 191 слоя когда начинается block5
# к сожалению на кагл нет возможности представить визуализацию нейросети с зумом, 
# чтобы показать вам замороженные слои и где проходит примерно половина этих слоев
# но это можно сделать по этой [ссылке](https://dgschwend.github.io/netscope/quickstart.html) 
# желательно перед заморозкой посмотреть на модель, 
# чтобы не провести линию заморозки между bn и activation
count_l = 0
for layer in base_model.layers:
    if (count_l >= 191) and (not isinstance(layer, BatchNormalization)): 
        layer.trainable = True
    count_l += 1

In [ ]:
# назначаем новые гиперпараметры для компиляции модели
print('Назначаем новые гиперпараметры компиляции модели для Fine-Tuning (Этап 2)')
C_LR = utils.hyperp('C_LR', 1e-4, '',results_of_exp,descr_hyperp_of_exp)
# уменьшаем шаг

In [ ]:
# Собираем модель 
model = utils.model_assembler(base_model, 
                              head)

# Компилируем модель
model.compile(loss=C_LOSS_TYPE, 
              optimizer=optimizers.Adam(lr=C_LR, 
                                        amsgrad=True), 
              metrics=['accuracy'])

In [ ]:
# выводим краткое описание модели по параметрам и слоям
utils.model_summary_short(model, 
                          base_model,
                          ' Fine-Tuning (Этап 2)')

In [ ]:
# инициализируем cb переменную и собираем списки cb 
time_cb = utils.TimingCallback()
callbacks_list = utils.callbacks_assembler(M_CALLBACKS_TYPE, 
                                           C_LR, 
                                           M_LR_UPDATE, 
                                           M_EPOCHS_DROP, 
                                           time_cb)

In [ ]:
# загружаем сохраненную на 1 Этапе модель и проверяем точность
print('Загружаем сохраненную на 1 Этапе модель и проверяем точность')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
# назначаем новые гиперпараметры для модели и обучения
print('Назначаем новые гиперпараметры для модели и обучения для Fine-Tuning (Этап 2)')
M_EPOCHS = utils.hyperp('M_EPOCHS', 12, '',results_of_exp,descr_hyperp_of_exp)

In [ ]:
# код обучения закомментирован для удобства воспроизведения ноутбука
# ниже идет полная загрузка ранее сохраненной модели этого обучения
# и выводится картинка с прошлым процессом обучения для наглядности
# если вы хотите выполнить обучение модели, закомментируйте код загрузки модели ниже

# # обучение модели для Fine-Tuning (Этап 2)
# right_steps_per_epoch = len(train_generator)

# history = model.fit_generator(train_generator,
#                               steps_per_epoch = right_steps_per_epoch,
#                               validation_data = valid_generator, 
#                               validation_steps = len(valid_generator),
#                               epochs = M_EPOCHS,
#                               callbacks = callbacks_list)

In [ ]:
# загрузка ранее выполненной и сохраненной модели Этапа 2 Fine-Tuning
print('Загружена ранее сохраненная лучшая из обученных моделей Fine-Tuning (Этап 2)')
model.load_weights('../input/last-result/best_model_42_20201225__15_54.hdf5')
utils.show_image('../input/last-result/pic_procces_model_fit_ft2_1.png',
                'Скриншоты обучения сохраненной модели Этапа 2 Fine-tuning', 
                 14, 12, 0.90)
utils.show_image('../input/last-result/pic_procces_model_fit_ft2_2.png',
                '', 14, 12, 0.9)
history, time_cb = [], []
model.save('best_model.hdf5')

In [ ]:
# загружаем лучшую модель и получаем значения метрик на валидационной выборке для результатов
print('Загружаем лучшую модель после обучения и проверяем метрики на валидационной выборке:')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
print('Сохраняем оценку метрики точности и значение функции потерь на лучшей моделе')
R_EVA_VAL_ACC = utils.hyperp('R_EVA_VAL_ACC', 
                             temp[1], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
R_EVA_VAL_LOSS = utils.hyperp('R_EVA_VAL_LOSS', 
                             temp[0], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)

In [ ]:
# сохраняем результаты обучения модели и всю лучшую модель
print('Cохраняем лучшую модель, ее параметры и результаты обучения:')
PATH_to_LAST_results_data = utils.save_model(PATH_to_WORKDIR,
                                             PATH_to_WORKDIR,
                                             model,
                                             history, 
                                             time_cb, 
                                             results_of_exp,
                                             descr_hyperp_of_exp)

In [ ]:
# рисуем графики точности и функции потерь по эпохам
# для удобства на графике номер экперимента, дата, время обучения и оценочная точность по вал выборке
utils.plot_acc_loss_fit_model_in_one (42, 
                                      results_of_exp,
                                      ' F-Tuning (Этап 2)')

In [ ]:
# выводим сводную информацию по гиперпараметрам и результатам 
utils.show_result_exp(42,
                      results_of_exp,
                      descr_hyperp_of_exp,
                      ' F-T (Этап 2)')

***Резюме по Fine-Tuning Этап 2:***
- По графику точности валидационной выборки можно сделать вывод, что кол-во эпох подобрано не оптимально так как точность и потери начали набирать темп. Наверняка стоило дообучить до 15 эпох и посмотреть что там, но мне было лень делать еще один эксперимент, потому что я заметил это только на этапе написания резюме.
- Среднее время выполнения пакета batch после разморозки половины базовой модели выросло на **27%** с 2.35 сек. до 3, среднее время выполнения эпохи выросло на **7%** с 479 до 509 сек. При этом общее время выполнения обучения сократилось до 6109 сек. с 7192, это связано с тем что колво эпох сократилось с 15 до 12. Увеличение среднего времени на эпохе всего на 7% очень вдохновляет, хотя кол-во тренируемых параметров выросло с 198154 до 10,021,992. Я это связываю с тем, что все-таки тренировка с нуля более трудоемкая операция и это видно на этом примере, что собственно еще раз подтверждает эффективность метода переноса обучения (Transfer learning)
- Этап 2 вырастил VAL_ACC на **12%** с 0.8642 до 0.9636 и снизило VAL_LOSS на **67%** с 0.6523 до 0.2123
- Шаг обучения подобран оптимально
- Проверил увеличение плато фиксированного LR до 2, но это не повлияло на результаты обучения
- sheduler показал оптимальный результат по формуле (LR * math.pow(LR_UPDATE, math.floor((1+epoch)/EPOCHS_DROP)) и при значении LR_UPDATE равном ***е*** в степени минус одна десятая

## 7.3 Этап 3. Разморозка всех слоев базовой модели

In [ ]:
# инициализируем новый эксперимент копированием гаперпараметров из предыдущего
# так как сесия не останавливалась
print('Инициализируем новый эксперимент Fine-Tuning (Этап 3):')
NUM_EXP = utils.new_exp_without_stop_session(results_of_exp)

In [ ]:
# размораживаем ВСЮ базовую модель
for layer in base_model.layers:
    if not isinstance(layer, BatchNormalization): 
        layer.trainable = True

In [ ]:
# назначаем новые гиперпараметры для компиляции модели
print('Назначаем новые гиперпараметры компиляции модели для Fine-Tuning (Этап 3)')
C_LR = utils.hyperp('C_LR', 
                    1e-5, 
                    '',
                    results_of_exp,
                    descr_hyperp_of_exp)
# уменьшаем шаг

In [ ]:
# Собираем модель 
model = utils.model_assembler(base_model, 
                              head)

# Компилируем модель
model.compile(loss=C_LOSS_TYPE, 
              optimizer=optimizers.Adam(lr=C_LR, 
                                        amsgrad=True), 
              metrics=['accuracy'])

In [ ]:
# выводим краткое описание модели по параметрам и слоям
utils.model_summary_short(model, 
                          base_model,
                          ' Fine-Tuning (Этап 3)')

In [ ]:
# инициализируем cb переменную и собираем списки cb 
time_cb = utils.TimingCallback()
callbacks_list = utils.callbacks_assembler(M_CALLBACKS_TYPE, 
                                           C_LR, 
                                           M_LR_UPDATE, 
                                           M_EPOCHS_DROP, 
                                           time_cb)

In [ ]:
# загружаем сохраненную на 2 Этапе модель и проверяем точность 
print('Загружаем сохраненную на 2 Этапе модель и проверяем точность')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)[1]

In [ ]:
# код обучения закомментирован для удобства воспроизведения ноутбука
# ниже идет полная загрузка ранее сохраненной модели этого обучения
# и выводится картинка с прошлым процессом обучения для наглядности
# если вы хотите выполнить обучение модели, закомментируйте код загрузки модели ниже

# # обучение модели Fine-Tuning (Этап 3)
# right_steps_per_epoch = len(train_generator)
# 
# history = model.fit_generator(train_generator,
#                               steps_per_epoch = right_steps_per_epoch,
#                               validation_data = valid_generator, 
#                               validation_steps = len(valid_generator),
#                               epochs = M_EPOCHS,
#                               callbacks = callbacks_list)

In [ ]:
# загрузка ранее выполненной и сохраненной модели Этапа 3 Fine-Tuning
print('Загружена ранее сохраненная лучшая из обученных моделей Fine-Tuning (Этап 3)')
model.load_weights('../input/last-result/best_model_51_20201228__04_27.hdf5')
utils.show_image('../input/last-result/pic_procces_model_fit_ft3_1.png',
                'Скриншот обучения сохраненной модели Этапа 3 Fine-tuning', 
                 14, 16, 0.95)
history, time_cb = [], []
model.save('best_model.hdf5')

In [ ]:
# загружаем лучшую модель и получаем значение оценки точности на валидационной выборке для результатов
print('Загружаем лучшую модель после обучения и проверяем метрики на валидационной выборке:')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
print('Сохраняем оценку метрики точности и значение функции потерь на лучшей моделе')
R_EVA_VAL_ACC = utils.hyperp('R_EVA_VAL_ACC', 
                             temp[1], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
R_EVA_VAL_LOSS = utils.hyperp('R_EVA_VAL_LOSS', 
                             temp[0], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)

In [ ]:
# сохраняем результаты обучения модели и всю лучшую модель
print('Cохраняем лучшую модель, ее параметры и результаты обучения:')
PATH_to_LAST_results_data = utils.save_model(PATH_to_WORKDIR,
                                             PATH_to_WORKDIR,
                                             model,
                                             history, 
                                             time_cb, 
                                             results_of_exp,
                                             descr_hyperp_of_exp)

In [ ]:
# рисуем графики точности и функции потерь по эпохам
# для удобства на графике номер экперимента, дата, время обучения и оценочная точность по вал выборке
utils.plot_acc_loss_fit_model_in_one (51, 
                                      results_of_exp,
                                      ' F-Tuning (Этап 3)')

In [ ]:
# выводим сводную информацию по гиперпараметрам и результатам 
utils.show_result_exp(51,
                      results_of_exp,
                      descr_hyperp_of_exp,
                      ' F-T (Этап 3)')

***Резюме по Fine-Tuning Этап 3:***
- Кол-во эпох подобрано оптимально (на последней эпохе остановка из-за отсутствия роста на 4 эпохах)
- Этап 3 вырастил VAL_ACC на **0.13%** с 0.9636 до 0.9689 и снизило VAL_LOSS на **9%** с 0.2123 до 0.1932
- Среднее время выполнения по сравнению с предыдущим этапом: пакета batch после полной разморозки базовой модели выросло на **15%** с 3 сек. до 3.45, среднее время выполнения эпохи выросло на **22%** с 509 до 626 сек, общее время выполнения обучения сократилось до 5011 с 6109, но это было связано к 8 эпохами из-за раннего стопа по Callback
- Шаг обучения подобран оптимально
- Проверил увеличение плато фиксированного LR до 2, но это не повлияло на результаты обучения
- sheduler показал оптимальный результат по формуле (LR * math.pow(LR_UPDATE, math.floor((1+epoch)/EPOCHS_DROP)) и при значении LR_UPDATE равном е в степени минус одна десятая

## 7.4 Визуализация результатов Fine-Tuning

In [ ]:
# выводим график сравнения точности и функции потерь 
# по разным этапам тонкой настройки
utils.plot_res_dif_exp_in_one([36,42,51],
                              ['freeze\nbase model\n100%', 
                               'freeze\nbase model\n50%', 
                               'freeze\nbase model\n0%'],
                              results_of_exp,
                              0.4)

# 8. Дообучение модели на увеличенном размере изображения (Extra Fit)
## 8.1 Создание модели и обучение

In [ ]:
# инициализируем новый эксперимент копированием гаперпараметров из предыдущего
# так как сесия не останавливалась
print('Инициализируем новый эксперимент Extra Fit (size+):')
NUM_EXP = utils.new_exp_without_stop_session(results_of_exp)

In [ ]:
# назначаем новые гиперпараметры для аугментации исходя из оптимальных
# при повторном изменении константы необязательно указывать описание гиперпараметра 
# оно уже сохранено в словаре при инициализации
print('Назначаем новые гиперпараметры для аугментации исходя из оптимальных для Extra Fit (size+)')
AUG_IMG_SIZE = utils.hyperp('AUG_IMG_SIZE', 
                            520, 
                            '',
                            results_of_exp,
                            descr_hyperp_of_exp)
AUG_BATCH_SIZE = utils.hyperp('AUG_BATCH_SIZE', 
                              12, 
                              '',
                              results_of_exp,
                              descr_hyperp_of_exp)
# пришлось уменьшить batch до максимально возможного чтобы уложиться в 16ГБ Tesla P100

In [ ]:
# генерируем новые аугментированные данные
train_generator, \
valid_generator, \
test_generator = utils.train_valid_test_generators(
                    AUG_ROTATION_RANGE,           
                    AUG_BRIGHTNES_RANGE,
                    AUG_WIDTH_SHIFT_RANGE, 
                    AUG_HEIGHT_SHIFT_RANGE,
                    AUG_HORIZONTAL_FLIP,
                    AUG_RESCALE,
                    AUG_VAL_SPLIT,
                    PATH_to_IMAGES_TRAIN,
                    PATH_to_IMAGES_TEST,
                    df_submit,
                    AUG_IMG_SIZE,
                    AUG_BATCH_SIZE,
                    RANDOM_SEED)

In [ ]:
# загружаем базовую модель заново на новых размерах фото
base_model = EfficientNetB3(
                    weights='imagenet', 
                    include_top=False, 
                    input_shape = (AUG_IMG_SIZE, 
                                   AUG_IMG_SIZE, 
                                   3))

In [ ]:
# назначаем новые гиперпараметры для головы и компиляции модели
print('Назначаем новые гиперпараметры компиляции модели для Extra Fit (size+)')
C_LR = utils.hyperp('C_LR', 
                    1e-4, 
                    '',
                    results_of_exp,
                    descr_hyperp_of_exp)
# увеличиваем шаг так как эксперимент показал что на шаге с 3 этапа модель 
# не чувствительна и не успевает обучится за 12 эпох

In [ ]:
# архитектура головы (классификационные слои поверх базовой модели)
head = S([GlobalAveragePooling2D(), 
          Dense(128, use_bias=H_USE_BIAS, kernel_regularizer=H_KERNEL_REG),
          BatchNormalization(axis=1), 
          Activation('relu'),
          Dropout(H_DROPOUT_RATE),
          Dense(10, activation='softmax')])


# Собираем модель используя подход перенос обучения (Transfer learning)
model = utils.model_assembler(base_model, head)

# Компилируем модель
model.compile(loss=C_LOSS_TYPE, 
              optimizer=optimizers.Adam(lr=C_LR, 
                                        amsgrad=True), 
              metrics=['accuracy'])

In [ ]:
# выводим краткое описание модели по параметрам и слоям
utils.model_summary_short(model, 
                          base_model,
                          ' Extra Fit (size+)')

In [ ]:
# загружаем сохраненную на 3 Этапе модель и проверяем точность 
print('Загружаем сохраненную на 3 Этапе модель и проверяем точность')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
# увеливаем плато фиксированного LR, чтобы модель успела обучиться за две эпохи
print('Назначаем новые гиперпараметры для модели и обучения для Extra Fit (size+)')
M_EPOCHS_DROP = utils.hyperp('M_EPOCHS_DROP', 
                             2, '',
                             results_of_exp,
                             descr_hyperp_of_exp)

In [ ]:
# инициализируем cb переменную и собираем списки cb 
time_cb = utils.TimingCallback()
callbacks_list = utils.callbacks_assembler(M_CALLBACKS_TYPE, 
                                           C_LR, 
                                           M_LR_UPDATE, 
                                           M_EPOCHS_DROP, 
                                           time_cb)

In [ ]:
# код обучения закомментирован для удобства воспроизведения ноутбука
# ниже идет полная загрузка ранее сохраненной модели этого обучения
# и выводится картинка с прошлым процессом обучения для наглядности
# если вы хотите выполнить обучение модели, закомментируйте код загрузки модели ниже

# # обучение модели для Extra Fit (size+)
# right_steps_per_epoch = len(train_generator)
# 
# history = model.fit_generator(train_generator,
#                               steps_per_epoch = right_steps_per_epoch,
#                               validation_data = valid_generator, 
#                               validation_steps = len(valid_generator),
#                               epochs = M_EPOCHS,
#                               callbacks = callbacks_list)

In [ ]:
# загрузка ранее выполненной и сохраненной модели на увелич изображении 520
print('Загружена ранее сохраненная лучшая из обученных моделей Extra Fit (size+)')
model.load_weights('../input/last-result/best_model_56_20201231__11_36.hdf5')
utils.show_image('../input/last-result/pic_procces_model_fit_520_1.png',
                'Скриншот обучения сохраненной модели на увелич. изображении', 
                 14, 15, 0.95)
utils.show_image('../input/last-result/pic_procces_model_fit_520_2.png',
                '', 
                 14, 15, 0.95)
history, time_cb = [], []
model.save('best_model.hdf5')

In [ ]:
# загружаем лучшую модель и получаем значение оценки точности на валидационной выборке для результатов
print('Загружаем лучшую модель после обучения и проверяем метрики на валидационной выборке:')
model.load_weights('best_model.hdf5')
temp = model.evaluate_generator(valid_generator, verbose=1)

In [ ]:
print('Сохраняем оценку метрики точности и значение функции потерь на лучшей моделе')
R_EVA_VAL_ACC = utils.hyperp('R_EVA_VAL_ACC', 
                             temp[1], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)
R_EVA_VAL_LOSS = utils.hyperp('R_EVA_VAL_LOSS', 
                             temp[0], 
                             '',
                             results_of_exp,
                             descr_hyperp_of_exp)

In [ ]:
# сохраняем результаты обучения модели и всю лучшую модель
print('Cохраняем лучшую модель, ее параметры и результаты обучения:')
PATH_to_LAST_results_data = utils.save_model(PATH_to_WORKDIR,
                                             PATH_to_WORKDIR,
                                             model,
                                             history, 
                                             time_cb, 
                                             results_of_exp,
                                             descr_hyperp_of_exp)

In [ ]:
# рисуем графики точности и функции потерь по эпохам
# для удобства на графике номер эксперимента, дата, время обучения и оценочная точность по вал выборке
utils.plot_acc_loss_fit_model_in_one (56, 
                                      results_of_exp,
                                      ' Extra Fit (size+)')

In [ ]:
# выводим сводную информацию по гиперпараметрам и результатам 
utils.show_result_exp(56,
                      results_of_exp,
                      descr_hyperp_of_exp,
                      ' Extra Fit')

## 8.2 Резюме по дообучению на увеличенном размере
- Дообучение вырастило VAL_ACC на **0.52%** с 0.969 до 0.9741 и снизило VAL_LOSS на **15%** с 0.1932 до 0.1698
- Кол-во эпох подобрано оптимально (на последней эпохе остановка из-за отсутсвия роста на 4 эпохах)
- Увеличение плато фиксированного LR положительно повлияло на обучение, так как именно на втором круге достигались оптимальные значения
- Общее время обучения модели составило 38152 сек. (=7192+6109+5011+19840). Это примерно 10.5 часов.

# 9. Формируем файл с предсказанием на тесте
#### 9.1. Предсказание без TTA

In [ ]:
test_generator.samples

In [ ]:
test_generator.reset()
predictions = model.predict_generator(test_generator, 
                                      steps=len(test_generator), 
                                      verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 
                           'Category':predictions}, 
                          columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_56.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

## 9.2. Предсказание c TTA (Test time augmentation)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# генерируем новые аугментированные данные
test_datagen = ImageDataGenerator(rotation_range=5, 
                                  brightness_range=[0.5, 2.5], 
                                  width_shift_range=AUG_WIDTH_SHIFT_RANGE, 
                                  height_shift_range=AUG_HEIGHT_SHIFT_RANGE, 
                                  horizontal_flip=AUG_HORIZONTAL_FLIP)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_submit, 
    directory=PATH_to_IMAGES_TEST, 
    x_col='Id', 
    y_col=None, 
    target_size=(AUG_IMG_SIZE, AUG_IMG_SIZE), 
    batch_size=AUG_BATCH_SIZE, 
    class_mode=None, 
    shuffle=False, 
    seed=RANDOM_SEED
    )

In [ ]:
NUM_PREDICTIONS_FOR_TTA = M_EPOCHS   
predictions_TTA = []

for i in range(NUM_PREDICTIONS_FOR_TTA):
    temp_predict = model.predict_generator(test_generator, verbose=1)
    predictions_TTA.append(temp_predict)

final_TTA_prediction = np.mean(np.array(predictions_TTA), axis=0)

In [ ]:
final_TTA_prediction[:2]

In [ ]:
predictions = np.argmax(final_TTA_prediction, axis=-1)
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 
                           'Category':predictions}, 
                          columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_56_TTA.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

# 10. Результаты:  
- score на kaggle = 0.97558 (7 место, Top 6%)  
- предварительные параметры для FT (EfficientNetB3, size=320, batch=32, Custom Head(128, Batch Normalization, use_bias=False, callback(MC_ES_LRS_T), остальные параметры лучше посмотреть в ноутбуке)
- дообучение на увеличенном размере фото (size=520, batch=**12**) вырастило VAL_ACC на 0.52% с 0.969 до 0.9741
- score без TTA - 0.97423
- score c TTA - 0.97558 (+0.13%)
- посмотреть этапы выполнения проекта с таймингом можно на [github](https://github.com/alex-sokolov2011/skillfactory_rds/tree/master/module_7). Там же:
  - результаты всех экспериментов в виде Pandas-датасета 
  - исходник файла utils_module09122020 со всеми утилитами
- к сожалению лучшие модели не удалось сохранить на гитхаб из-за их объема, но при желании вы можете скачать их из открытого [датасета](https://www.kaggle.com/sokolovaleks/last-result) 

К сожалению не хватило времени, чтобы:  
- вычислить на каких картинках ошибается модель
- попытаться дополнить этот класс фото извне внешним датасетом
- дообучить модель на сложных примерах (в том числе: 
  - 7 кат. - Лада 2109, а 9 кат. - Лада 21099, которые отличаются только типом кузова сзади. Спереди эти модели выпускаются на эдентичных запчастях. Поэтому фото спереди по этим моделям не отличимы в принципе. 
  - первая фото в тесте 305108.jpg относится к кат. 2 - Lada Samara 2114 из-за наличия молдинга. Но так как фото сделано в темное время суток настроек аугментации не хватает чтобы выкрутить яркость и она классифицируется как 7 и 9 кат. Можно было бы сделать промежуточную сеть, чтобы классифицировать темные изображения и доучить на них модель или отдельно проклассифицировать, но это уже будет не Single Model.
- организовать проверку выбросов в тренировочной выборке с помощью обученной модели (я там краем глаза видел грузовик :) когда просматривал фото)

# 11. Очистка диска перед RunAndSave

In [ ]:
# чистим рабочую папку перед сохранением  
utils.clean_HDD_befor_RUN_and_SAVE(PATH_to_WORKDIR,
                           [PATH_to_IMAGES], 
                            ['best_model.hdf5'], 
                            ['.zip'])

# I hope you like my kernel. Thanks for upvote
Надеюсь ноутбук вам понравился. Не стесняйтесь писать вопросы лично и в комментариях. Также буду рад любым замечаниям и предложениям по улучшению ноута. 

Ну и если ноутбук показался вам интересным или полезным, то я буду благодарен вам за апвойт[1]

[1] ***Апвойт - кнопка со стрелкой вверх чуть ниже этих строк под горизонтальной линией :)***